In [1]:
library(devtools)
library(EWCE)
library(ggplot2)
library(cowplot)
library(limma)
library(readxl)

Loading required package: usethis


In [6]:
ctd <- EWCE::ctd()

ERROR: Error in eval(expr, envir, enclos): attempt to apply non-function


In [2]:
setwd("~/ownCloud/Meng/")

In [3]:
## Cell type enrichment analysis start

In [4]:
EWCE_specificity_matrix <- read.table("Cell_type_cell_markers/EWCE_Darm&Hodge_specificitymatrix.csv",
                                      header =TRUE, stringsAsFactors=F, sep = ",")
rownames(EWCE_specificity_matrix) <- EWCE_specificity_matrix$Genes
EWCE_specificity_matrix <- EWCE_specificity_matrix[,-1]

In [5]:
sct.data <- NULL

sct.data[[1]] <- list(annot=NULL, mean_exp = EWCE_specificity_matrix,
                      median_exp=EWCE_specificity_matrix,
                      specificity=EWCE_specificity_matrix, median_specificity=EWCE_specificity_matrix,
                      specificity_quantiles=EWCE_specificity_matrix, plotting=NULL)

reps=20000 # <- Use 100 bootstrap lists so it runs quickly, for publishable analysis use >10000
level=1 # <- Use level 1 annotations (i.e. Interneurons)

In [6]:
background <- read.table("100_plus_results/Protein basic info.tsv", 
                         sep = "\t", header =TRUE, stringsAsFactors=F)

In [57]:
P_age <- read.table(paste("100_plus_results/Age_related_protein/Age_protein_", 114, ".txt", sep = ""),
                            header =TRUE,
                            stringsAsFactors=F, sep = "\t")
P_age_down <- P_age[which(P_age$slope < 0), ]
P_age_up <- P_age[which(P_age$slope > 0), ]

In [48]:
P_age_down

,protein,genesymbol,HUGO_gene,slope
,<chr>,<chr>,<chr>,<dbl>
1,SLC12A2,SLC12A2,SLC12A2,-0.016475472
2,MAP1B,MAP1B,MAP1B,-0.007200298
3,IRGQ,IRGQ,IRGQ,-0.012766274
5,HSPA12A,HSPA12A,HSPA12A,-0.005648299
6,ATP1A2,ATP1A2,ATP1A2,-0.005950950
7,ATP1A1;ATP1A3,ATP1A1,ATP1A1,-0.005284961
8,SHANK2,SHANK2,SHANK2,-0.011159894
9,HADHA,HADHA,HADHA,-0.006187667
12,RPS3,RPS3,RPS3,-0.003474713


In [49]:
EWCE_specificity_matrix

,Astrocytes,Endothelial,ExNeurons,InNeurons,Microglia,Oligodendrocytes,OPC,Unknown
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
X3.8.1.2,0.00000000,0.000000000,0.06802946,0.931942533,0.00000000,0.000000000,0.000000000,0.0000000000
X3.8.1.3,0.87848240,0.000000000,0.07530923,0.046207127,0.00000000,0.000000000,0.000000000,0.0000000000
X3.8.1.4,0.00000000,0.000000000,0.24498483,0.755006814,0.00000000,0.000000000,0.000000000,0.0000000000
X3.8.1.5,0.00000000,0.000000000,0.63114994,0.097241006,0.00000000,0.271498008,0.000000000,0.0000000000
X5.HT3C2,0.06982341,0.000000000,0.18086182,0.225758009,0.01349505,0.037695567,0.472365751,0.0000000000
A1BG,0.01156738,0.000000000,0.20158860,0.125844117,0.10621237,0.553589011,0.001198491,0.0000000000
A1BG.AS1,0.07093724,0.000000000,0.36894088,0.241280229,0.02195090,0.295645028,0.001245377,0.0000000000
A1CF,0.09884967,0.000000000,0.22841492,0.130812357,0.06467911,0.051763628,0.425480171,0.0000000000
A2M,0.03423939,0.189783442,0.01314316,0.009476242,0.71128111,0.018847035,0.023229623,0.0000000000


In [58]:
EWCE_specificity_matrix[rownames(EWCE_specificity_matrix) %in% P_age_up$HUGO_gene, ]

,Astrocytes,Endothelial,ExNeurons,InNeurons,Microglia,Oligodendrocytes,OPC,Unknown
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
ABLIM1,0.36649692,0.126405692,0.130528784,0.227985307,0.037770375,0.023634059,0.064725775,2.245309e-02
AMPD2,0.19766488,0.023119134,0.130744075,0.229675964,0.156538075,0.077315937,0.158922675,2.601925e-02
ATP8A1,0.10416472,0.028964078,0.180143588,0.145403926,0.063908906,0.425898058,0.051364796,1.519238e-04
CD44,0.49829421,0.002444542,0.008869418,0.107352584,0.098468983,0.001203878,0.161982148,1.213842e-01
CHDH,0.30301537,0.007868731,0.008993446,0.021544071,0.068216602,0.066364518,0.520767998,3.229261e-03
CIT,0.04643347,0.042662069,0.287976811,0.287346253,0.061705767,0.050760989,0.185027338,3.808731e-02
ENTPD3,0.05449619,0.122188572,0.110820045,0.593918538,0.039372112,0.042320620,0.034763754,2.120162e-03
GGA3,0.11371456,0.043331695,0.168778572,0.138858676,0.266842285,0.113139153,0.144044844,1.129021e-02
ILF2,0.07168092,0.065909532,0.077309606,0.097092773,0.047855495,0.145826088,0.161736445,3.325891e-01


In [60]:
colSums(EWCE_specificity_matrix[rownames(EWCE_specificity_matrix) %in% P_age_up$HUGO_gene, ])/colSums(EWCE_specificity_matrix[rownames(EWCE_specificity_matrix) %in% background$HUGO_gene, ])

Astrocytes      Endothelial        ExNeurons        InNeurons 
     0.013510773      0.008234920      0.011012842      0.010491736 
       Microglia Oligodendrocytes              OPC          Unknown 
     0.009387224      0.006299626      0.008379579      0.010239940

In [55]:
colSums(EWCE_specificity_matrix[rownames(EWCE_specificity_matrix) %in% background$HUGO_gene, ])

Astrocytes      Endothelial        ExNeurons        InNeurons 
        391.8259         147.5528         560.8466         520.2898 
       Microglia Oligodendrocytes              OPC          Unknown 
        340.1058         426.8627         389.5119         167.0045

In [9]:
num_cluster = 6

In [12]:
for(i in 1:num_cluster){

    set.seed(1234)
    P_cluster <- read.table(paste("100_plus_results/Protein_cluster/Braak_protein_cluster_", i, ".tsv", sep = ""),
                            header =TRUE,
                            stringsAsFactors=F, sep = "\t")
    
    full_results = bootstrap.enrichment.test(sct_data=sct.data, 
                                             hits=P_cluster$HUGO_gene, bg = background$HUGO_gene, 
                                             genelistSpecies="human",sctSpecies="human",
                                             reps=reps, annotLevel=level)
    
    full_results$results
    
    write.table(full_results$results,
                paste("100_plus_results/Cell_type_deconvolution/EWCE_tempcort_ProtCluster", i, 
                      "_BasedonDarm&Hodge.csv", sep = ''), sep = ',',
                row.names = FALSE, col.names = TRUE, quote = FALSE)
}

 [1] "NDUFS3"   "PDK3"     "NDUFA12"  "SLC25A11" "NDUFS8"   "NDUFS1"  
 [7] "SLC25A4"  "NDUFA13"  "NDUFS2"   "NDUFA5"   "NNT"      "LRPPRC"  
[13] "NDUFB3"   "APOO"     "COX4I1"   "NDUFA11"  "NDUFS7"   "SLC25A4" 
[19] "SLC25A5"  "NDUFA9"   "FXYD7"    "NDUFAF4"  "NDUFB8"   "ACTN1"   
[25] "SLC25A27" "MTX2"     "NDUFB4"   "NDUFB9"   "GABRB1"   "NDUFA2"  
[31] "GK"       "COX7A2L"  "TMX3"     "SLC25A3"  "NDUFC2"   "TMEM177" 
[37] "SFXN4"    "FBXO41"   "AP3M1"    "SCN1A"    "NDUFB5"   "DIP2A"   
[43] "BDH1"     "MICU3"    "RPL15"    "DLAT"     "VDAC1"    "PDHB"    
[49] "NDUFB10"  "HSPA12A"  "SLC25A5"  "ATP1A1"   "NDUFA10"  "SFXN1"   
[55] "SLC25A4"  "ATP2B2"   "RAP2A"    "TMEM126A" "AGK"      "MGST3"   
[61] "NDUFA6"   "UQCR10"   "COX5A"    "PRKCB"    "FSD1"     "BRSK1"   
[67] "NDUFA4"   "BRSK2"    "KIAA1586" "AP2M1"    "COX7A2"   "SPG7"    
[73] "RPL5"     "NDUFB1"   "SLC25A12" "MPC2"     "PLXNA2"   "HCN2"    
[79] "CEND1"    "C6orf136" "CYB5R1"   "GABRG1"   "ABHD16A" 
[1] "Astrocytes"


In [21]:
for (i in c(60, 114)){

    set.seed(1234)
    P_age <- read.table(paste("100_plus_results/Age_related_protein/Age_protein_", i, ".txt", sep = ""),
                            header =TRUE,
                            stringsAsFactors=F, sep = "\t")
    P_age_down <- P_age[which(P_age$slope < 0), ]
    print(dim(P_age_down))
    P_age_up <- P_age[which(P_age$slope > 0), ]
    print(dim(P_age_up))
    
    full_results = bootstrap.enrichment.test(sct_data=sct.data, 
                                             hits=P_age_down$HUGO_gene, bg = background$HUGO_gene, 
                                             genelistSpecies="human",sctSpecies="human",
                                             reps=reps, annotLevel=level)
    #write.table(full_results$results,
    #            paste("100_plus_results/Cell_type_deconvolution/EWCE_tempcort_AgeProt", i, 
    #                  "_DOWN_BasedonDarm&Hodge.csv", sep = ''), sep = ',',
    #            row.names = FALSE, col.names = TRUE, quote = FALSE)
    
    full_results = bootstrap.enrichment.test(sct_data=sct.data, 
                                             hits=P_age_up$HUGO_gene, bg = background$HUGO_gene, 
                                             genelistSpecies="human",sctSpecies="human",
                                             reps=reps, annotLevel=level)
    #write.table(full_results$results,
    #            paste("100_plus_results/Cell_type_deconvolution/EWCE_tempcort_AgeProt", i, 
    #                  "_UP_BasedonDarm&Hodge.csv", sep = ''), sep = ',',
    #            row.names = FALSE, col.names = TRUE, quote = FALSE)
}

[1] 41  4
[1] 19  4
 [1] "CARNS1"   "SLC16A1"  "NEFM"     "CNP"      "SNX9"     "MOG"     
 [7] "NEFL"     "HSP90AA1" "NEFH"     "RAB11A"   "STX5"     "SIRT2"   
[13] "SLC44A1"  "CD9"      "RPL9"     "PLEKHB1"  "C2"       "RPS16"   
[19] "NPC1"     "MAG"      "PIP4K2A"  "PDIA6"    "PNCK"     "CD82"    
[25] "SHMT2"    "CDK1"     "PIP4K2A"  "LGALS3BP" "INA"      "DDX49"   
[31] "ENPP6"    "UBAP2L"   "NAP1L1"   "PLP1"     "CAND1"    "ANLN"    
[37] "RPL31"    "INA"      "CLDN11"   "CHMP1A"  
[1] "Astrocytes"
[1] 1
[1] ""
[1] "Endothelial"
[1] 0.82025
[1] ""
[1] "ExNeurons"
[1] 1
[1] ""
[1] "InNeurons"
[1] 1
[1] ""
[1] "Microglia"
[1] 0.9504
[1] ""
[1] "Oligodendrocytes"
[1] 0
[1] "Fold enrichment: 2.35829110710454"
[1] "Standard deviations from mean: 10.7112469634987"
[1] ""
[1] "OPC"
[1] 0.9793
[1] ""
[1] "Unknown"
[1] 0.174
[1] ""
 [1] "FLOT1"   "KBTBD11" "AMPH"    "PTGES2"  "PDPK1"   "CACNA1E" "NCDN"   
 [8] "ICAM5"   "ATP2B4"  "DCLK2"   "PDE1A"   "PSD3"    "PUS1"    "OGFRL1" 
[15] "R

In [41]:
aging_protective_protein <- read.table("100_plus_results/Age_related_protein/Age_protein_CEN_specific_clean_38.tsv",
                                       header =TRUE,
                                       stringsAsFactors=F, sep = "\t")

In [42]:
aging_protective_protein

protein,genesymbol,HUGO_gene,celltype,p.vale,adj.lm.p,slope
<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>
CAND1;CAND2,CAND1,CAND1,NA,1.877166e-06,6.472468e-03,-0.011316704
NCDN,NCDN,NCDN,NA,3.319534e-07,1.144575e-03,0.007506020
CDK1,CDK1,CDK1,Unknown,4.068028e-06,1.402656e-02,-0.011084598
PNCK,PNCK,PNCK,NA,9.823276e-06,3.387066e-02,-0.030848688
NAP1L1,NAP1L1,NAP1L1,NA,7.609953e-06,2.623912e-02,-0.005102386
NCKAP1;NCKAP1L,NCKAP1,NCKAP1,NA,1.051173e-07,3.624444e-04,0.014418741
PIP4K2A;PIP4K2B,PIP4K2A,PIP4K2A,Oligodendrocytes,1.067889e-06,3.682081e-03,-0.004739831
CACNA1E,CACNA1E,CACNA1E,NA,9.939724e-08,3.427217e-04,0.014524662
C2;CFB,C2,C2,Microglia,3.030935e-09,1.045066e-05,-0.013279683


In [46]:
P_age_down <- aging_protective_protein[which(aging_protective_protein$slope < 0), ]
P_age_up <- aging_protective_protein[which(aging_protective_protein$slope > 0), ]

full_results = bootstrap.enrichment.test(sct_data=sct.data, 
                                         hits=P_age_down$HUGO_gene, bg = background$HUGO_gene, 
                                         genelistSpecies="human",sctSpecies="human",
                                         reps=reps, annotLevel=level)
write.table(full_results$results,
            paste("100_plus_results/Cell_type_deconvolution/EWCE_tempcort_AgeProtect", 38, 
                  "_DOWN_BasedonDarm&Hodge.csv", sep = ''), sep = ',',
            row.names = FALSE, col.names = TRUE, quote = FALSE)

full_results = bootstrap.enrichment.test(sct_data=sct.data, 
                                         hits=P_age_up$HUGO_gene, bg = background$HUGO_gene, 
                                         genelistSpecies="human",sctSpecies="human",
                                         reps=reps, annotLevel=level)
write.table(full_results$results,
            paste("100_plus_results/Cell_type_deconvolution/EWCE_tempcort_AgeProtect", 38, 
                  "_UP_BasedonDarm&Hodge.csv", sep = ''), sep = ',',
            row.names = FALSE, col.names = TRUE, quote = FALSE)

 [1] "CAND1"    "CDK1"     "PNCK"     "NAP1L1"   "PIP4K2A"  "C2"      
 [7] "CHMP1A"   "PLEKHB1"  "CARNS1"   "LGALS3BP" "SHMT2"    "ENPP6"   
[13] "SNX9"     "CD9"      "RAB11A"   "CNP"      "STX5"     "ANLN"    
[19] "PDIA6"    "MOG"      "UBAP2L"   "RPL31"    "NEFL"     "SIRT2"   
[25] "NEFM"     "MAG"      "SLC44A1" 
[1] "Astrocytes"
[1] 1
[1] ""
[1] "Endothelial"
[1] 0.9468
[1] ""
[1] "ExNeurons"
[1] 0.99915
[1] ""
[1] "InNeurons"
[1] 1
[1] ""
[1] "Microglia"
[1] 0.7122
[1] ""
[1] "Oligodendrocytes"
[1] 0
[1] "Fold enrichment: 2.52605666380465"
[1] "Standard deviations from mean: 9.90492886823605"
[1] ""
[1] "OPC"
[1] 0.82175
[1] ""
[1] "Unknown"
[1] 0.06155
[1] ""
 [1] "NCDN"    "NCKAP1"  "CACNA1E" "OGFRL1"  "PDPK1"   "PTGES2"  "FAM169A"
 [8] "DNAJB1"  "PDE1A"   "PSD3"   
[1] "Astrocytes"
[1] 0.54205
[1] ""
[1] "Endothelial"
[1] 0.3524
[1] ""
[1] "ExNeurons"
[1] 0.03365
[1] "Fold enrichment: 1.36869323770609"
[1] "Standard deviations from mean: 1.96523595579471"
[1] ""
[1] "InNeur

In [22]:
write.table(full_results$results,
            "100_plus_results/Cell_type_deconvolution/EWCE_tempcort_mitochondria_DOWN_BasedonDarm&Hodge.csv", 
            sep = ',',
            row.names = FALSE, col.names = TRUE, quote = FALSE)

In [10]:
proteins_list <- background$HUGO_gene[grep("^NDUF",  background$protein)]

In [11]:
proteins_list

[1] "NDUFV2"  "NDUFB10" "NDUFB9"  "NDUFA2"  "NDUFS1"  "NDUFS2"  "NDUFA3" 
 [8] "NDUFA11" "NDUFA8"  "NDUFA9"  "NDUFS3"  "NDUFB8"  "NDUFB7"  "NDUFAF4"
[15] "NDUFV1"  "NDUFC2"  "NDUFB1"  "NDUFA13" "NDUFS8"  "NDUFA6"  "NDUFB4" 
[22] "NDUFA12" "NDUFA4"  "NDUFB5"  "NDUFB3"  "NDUFAF3" "NDUFS5"  "NDUFA5" 
[29] "NDUFA10" "NDUFS7"  "NDUFS4"

In [12]:
mitochondria_proteins <- data.frame(protein = proteins_list, regulated = rep("Down", length(proteins_list)), 
                                    stringsAsFactors=F)

mitochondria_proteins[mitochondria_proteins$protein %in% c("NDUFS5", "NDUFS4"), ]$regulated <- "Up"

mitochondria_background_HUGO <- as.data.frame(read.table("100_plus_project/External_Resources/hgnc-symbol-check.csv",
                                                     stringsAsFactors=F, header = TRUE, sep = ","))

In [23]:
mitochondria_proteins

protein,regulated
<chr>,<chr>
NDUFV2,Down
NDUFB10,Down
NDUFB9,Down
NDUFA2,Down
NDUFS1,Down
NDUFS2,Down
NDUFA3,Down
NDUFA11,Down
NDUFA8,Down


In [15]:
length(intersect(mitochondria_background_HUGO$Approved.symbol, background$HUGO_gene))

[1] 492

In [20]:
full_results = bootstrap.enrichment.test(sct_data=sct.data, 
                                         hits=mitochondria_proteins[mitochondria_proteins$regulated == "Down", ]$protein, 
                                         bg=intersect(mitochondria_background_HUGO$Approved.symbol, background$HUGO_gene),
                                         genelistSpecies="human",sctSpecies="human",
                                         reps=reps, annotLevel=level)

 [1] "NDUFV2"  "NDUFB10" "NDUFB9"  "NDUFA2"  "NDUFS1"  "NDUFS2"  "NDUFA3" 
 [8] "NDUFA11" "NDUFA8"  "NDUFA9"  "NDUFS3"  "NDUFB8"  "NDUFB7"  "NDUFAF4"
[15] "NDUFV1"  "NDUFC2"  "NDUFB1"  "NDUFA13" "NDUFS8"  "NDUFA6"  "NDUFB4" 
[22] "NDUFA12" "NDUFA4"  "NDUFB5"  "NDUFB3"  "NDUFAF3" "NDUFA5"  "NDUFA10"
[29] "NDUFS7" 
[1] "Astrocytes"
[1] 0.8585
[1] ""
[1] "Endothelial"
[1] 0.2237
[1] ""
[1] "ExNeurons"
[1] 0.152
[1] ""
[1] "InNeurons"
[1] 0.01355
[1] "Fold enrichment: 1.14413678005438"
[1] "Standard deviations from mean: 2.21709047617746"
[1] ""
[1] "Microglia"
[1] 0.97345
[1] ""
[1] "Oligodendrocytes"
[1] 0.04145
[1] "Fold enrichment: 1.20276994853783"
[1] "Standard deviations from mean: 1.86670843098339"
[1] ""
[1] "OPC"
[1] 0.9991
[1] ""
[1] "Unknown"
[1] 0.3006
[1] ""


In [21]:
full_results

$results
                         CellType annotLevel       p fold_change sd_from_mean
Astrocytes             Astrocytes          1 0.85850   0.8589390   -1.0553692
Endothelial           Endothelial          1 0.22370   1.1488023    0.6808237
ExNeurons               ExNeurons          1 0.15200   1.0791577    1.0283285
InNeurons               InNeurons          1 0.01355   1.1441368    2.2170905
Microglia               Microglia          1 0.97345   0.7983993   -1.7774919
Oligodendrocytes Oligodendrocytes          1 0.04145   1.2027699    1.8667084
OPC                           OPC          1 0.99910   0.7580274   -2.7922677
Unknown                   Unknown          1 0.30060   1.1070404    0.4907659

$hit.cells
      Astrocytes      Endothelial        ExNeurons        InNeurons 
        3.950367         1.448023         5.349077         5.812334 
       Microglia Oligodendrocytes              OPC          Unknown 
        2.634135         4.998152         3.139561         1.668350 

$bootstrap_data
         Astrocytes Endothelial ExNeurons InNeurons Microglia Oligodendrocytes
    [1,]   4.666129   1.2795813  4.895888  5.338154  3.523455         3.813615
    [2,]   4.330102   0.9325218  5.065623  5.166903  2.959752         4.150901
    [3,]   4.572609   1.2012712  4.573959  4.910515  3.286271         4.369611
    [4,]   3.731322   1.1224516  4.771067  5.206374  3.665330         4.401261
    [5,]   5.047182   0.8171406  5.011019  4.983446  3.011313         4.301413
    [6,]   4.183685   1.4181280  5.785090  5.381284  3.131100         3.779491
    [7,]   4.154272   1.0175109  5.718186  5.313205  3.375594         3.949284
    [8,]   4.807952   1.1550302  5.032277  5.307674  3.487278         3.685802
    [9,]   4.028339   1.3505189  5.239252  5.297593  3.264281         4.898702
   [10,]   5.508422   1.3380558  4.084578  4.417662  3.739193         4.007556
   [11,]   5.658678   1.0722276  4.194337  4.624609  3.365326         3.552322
   [12,]   5.111928   1.1274612  5.312550  5.467765  2.942793         3.937554
   [13,]   3.684116   1.5830553  5.152686  5.381547  3.532942         3.645668
   [14,]   3.466634   1.2125713  5.156078  5.312919  3.628985         4.602738
   [15,]   4.106110   1.5383929  4.947427  5.193220  3.120055         4.321392
   [16,]   4.533755   0.9747126  5.702550  5.595411  2.679200         3.356548
   [17,]   4.803702   1.3679331  4.354891  4.477027  3.504987         3.901708
   [18,]   4.057461   1.1716834  5.491852  5.306209  3.896745         3.553937
   [19,]   4.585685   1.5535639  4.918903  4.991911  3.319949         4.181158
   [20,]   5.815786   1.0285157  4.690502  4.980769  3.022543         4.567978
   [21,]   4.438932   0.8725779  4.933204  5.464820  3.148998         4.030475
   [22,]   4.029119   1.6476752  4.840408  5.176842  3.150329         3.822383
   [23,]   3.926444   1.0037121  5.408933  5.974851  2.802051         4.325979
   [24,]   4.666916   0.8603841  5.407894  5.582616  3.216920         4.466648
   [25,]   4.060283   1.5513079  5.193198  5.411111  3.862478         3.617612
   [26,]   5.412140   1.4019240  4.549897  4.567100  3.615135         3.635081
   [27,]   4.838698   1.4161431  4.803891  4.731383  3.395208         3.962340
   [28,]   4.424686   0.8028343  5.526140  5.167994  3.636222         3.647451
   [29,]   3.974139   0.9704026  5.940064  5.655756  3.394183         4.476144
   [30,]   5.354624   1.0852707  4.776102  4.661952  3.618662         3.612537
   [31,]   5.102210   0.8973356  5.598921  5.207754  2.823191         4.504986
   [32,]   5.093324   0.8353708  4.998795  5.312832  3.848791         3.422463
   [33,]   3.346878   1.2791748  5.846053  5.347595  2.680154         4.452582
   [34,]   4.276991   1.5695016  5.041529  5.265641  3.240656         4.043691
   [35,]   4.578842   1.1804807  4.626583  5.283700  3.644855         4.006915
   [36,]   5.286520   1.3462468  4.576112  4.970805  3.430157         3.858000
   [37,]   4.967878   1.2375532  4.329501  4.621744  3.456092

In [7]:
ANOVA_lm_test_res <- read.table('100_plus_results/sig_protein_list/ANOVA_lm_test_results.tsv', 
                                 header =TRUE, stringsAsFactors=F, sep = "\t") 

In [9]:
mitochondria_background_HUGO <- as.data.frame(read.table("100_plus_project/External_Resources/hgnc-symbol-check.csv",
                                                     stringsAsFactors=F, header = TRUE, sep = ","))

In [17]:
regulated_mito_protein <- NULL
for(i in 1:nrow(background)){
    if(background[i,]$HUGO_gene %in% mitochondria_background_HUGO$Approved.symbol){
        if(ANOVA_lm_test_res[i, ]$adj.p_value_lm < 0.05){
            if(ANOVA_lm_test_res[i, ]$effect_size_lm < 0){
                regulated_mito_protein <- rbind(regulated_mito_protein, c(background[i,]$HUGO_gene, "Down"))
            }
            else{
                regulated_mito_protein <- rbind(regulated_mito_protein, c(background[i,]$HUGO_gene, "Up"))
            }
        }
    }
}

In [19]:
regulated_mito_protein <- as.data.frame(regulated_mito_protein, stringsAsFactors = FALSE)
colnames(regulated_mito_protein) <- c("protein", "regulated")

In [28]:
full_results = bootstrap.enrichment.test(sct_data=sct.data, 
                                         hits=regulated_mito_protein[regulated_mito_protein$regulated == "Up", ]$protein, 
                                         bg=intersect(mitochondria_background_HUGO$Approved.symbol, background$HUGO_gene),
                                         genelistSpecies="human",sctSpecies="human",
                                         reps=reps, annotLevel=level)

  [1] "FH"        "IDH3B"     "GRHPR"     "ALDH2"     "QDPR"      "HEBP1"    
  [7] "FASN"      "ACAA2"     "ACAD8"     "ACOT7"     "ETFA"      "EPHX2"    
 [13] "PAICS"     "ABHD10"    "HADHA"     "ALDH1L1"   "CAT"       "ACAA1"    
 [19] "RMDN1"     "ALDH6A1"   "MCCC2"     "CRYZ"      "HTRA2"     "HSD17B8"  
 [25] "GCDH"      "SOD2"      "GLUD1"     "FAHD2A"    "ABAT"      "GSR"      
 [31] "NIF3L1"    "ECHS1"     "NADK2"     "HSDL2"     "AKR7A2"    "DLD"      
 [37] "ALDH9A1"   "ACADM"     "TMLHE"     "MAOB"      "HADHB"     "MAOA"     
 [43] "PRDX2"     "TRNT1"     "HSD17B4"   "AUH"       "DCXR"      "ECI1"     
 [49] "AK3"       "IDH2"      "ALDH4A1"   "ME2"       "SLC25A20"  "IDH3A"    
 [55] "PREPL"     "SARS2"     "HIBADH"    "PRDX6"     "EFHD1"     "ACAT1"    
 [61] "SUCLG2"    "ALDH3A2"   "ACSS3"     "GRPEL1"    "ALDH7A1"   "SPR"      
 [67] "HINT3"     "ACSS1"     "ACOT9"     "ACADS"     "SND1"      "HADH"     
 [73] "HSD17B10"  "PARK7"     "HAGH"      "PNPO"      "BCL2L1"  

In [29]:
write.table(full_results$results,
            "100_plus_results/Cell_type_deconvolution/EWCE_regulated_mitochondria_Up_BasedonDarm&Hodge.csv", 
            sep = ',',
            row.names = FALSE, col.names = TRUE, quote = FALSE)

In [11]:
num_cluster = 4

for(i in 1:num_cluster){

    set.seed(1234)
    P_cluster <- read.table(paste("100_plus_results/Age_Protein_cluster/Age_protein_cluster_", i, ".tsv", sep = ""),
                            header =TRUE,
                            stringsAsFactors=F, sep = "\t")
    
    full_results = bootstrap.enrichment.test(sct_data=sct.data, 
                                             hits=P_cluster$HUGO_gene, bg = background$HUGO_gene, 
                                             genelistSpecies="human",sctSpecies="human",
                                             reps=reps, annotLevel=level)
    
    full_results$results
    
    write.table(full_results$results,
                paste("100_plus_results/Cell_type_deconvolution/EWCE_tempcort_Age_ProtCluster", i, 
                      "_BasedonDarm&Hodge.csv", sep = ''), sep = ',',
                row.names = FALSE, col.names = TRUE, quote = FALSE)
}

 [1] "MTHFD1"  "FLOT1"   "KBTBD11" "RPS6KA2" "CIT"     "TPD52L1" "PTGES2" 
 [8] "RRAGC"   "PDPK1"   "CACNA1E" "NCDN"    "NPTX1"   "CD44"    "ICAM5"  
[15] "RAB27B"  "ATP2B4"  "AMPD2"   "TMEM30A" "DCLK2"   "PTK2B"   "USP7"   
[22] "PTPRN"   "PDE1A"   "ABLIM1"  "RPS6KA2" "USP11"   "PSD3"    "PUS1"   
[29] "CHDH"    "LZTS1"   "GGA3"    "NUMA1"   "OGFRL1"  "SNX5"    "ATP8A1" 
[36] "RAB33B"  "PCSK1"   "ENTPD3"  "MAP3K5"  "RIMBP2"  "FAM169A" "SLC2A13"
[43] "SSB"     "ILF2"    "ITGB4"   "NPTXR"   "NCKAP1"  "DNAJB1" 
[1] "Astrocytes"
[1] 0.06235
[1] ""
[1] "Endothelial"
[1] 0.89735
[1] ""
[1] "ExNeurons"
[1] 0.01475
[1] "Fold enrichment: 1.19353298977837"
[1] "Standard deviations from mean: 2.28309133079372"
[1] ""
[1] "InNeurons"
[1] 0.3996
[1] ""
[1] "Microglia"
[1] 0.65445
[1] ""
[1] "Oligodendrocytes"
[1] 0.9994
[1] ""
[1] "OPC"
[1] 0.97345
[1] ""
[1] "Unknown"
[1] 0.6556
[1] ""
 [1] "SLC16A1"  "IRGQ"     "HSPA12A"  "ATP1A2"   "AMPH"     "ATP1A1"  
 [7] "SHANK2"   "HADHA"    "RPS3"     "RA

In [ ]:
## test and re-produce the result from Suzanne

In [39]:
cell_type_matrix <- read.table("R_workstation/100_plus_proteomic_cell_type_enrichment/EWCE_celltype.tsv",
                               header =TRUE,
                               stringsAsFactors=F, sep = "\t")

In [40]:
cell_type_matrix

protein,genesymbol,HUGO_gene,celltype
<chr>,<chr>,<chr>,<chr>
SLC12A2,SLC12A2,SLC12A2,Oligodendrocytes
CASKIN1,CASKIN1,CASKIN1,NA
GNG2,GNG2,GNG2,NA
CARNS1,CARNS1,CARNS1,Oligodendrocytes
FH,FH,FH,NA
EFHD2,EFHD2,EFHD2,NA
PCLO,PCLO,PCLO,NA
HSPA1A;HSPA1B,HSPA1A,HSPA1A,NA
STIP1,STIP1,STIP1,NA


In [41]:
cluster_1 <- read.table("100_plus_results/Protein_cluster/protein_cluster_1.txt",
                               header =TRUE,
                               stringsAsFactors=F, sep = "\t")

In [42]:
cluster_1

protein,gene,HUGO_gene
<chr>,<chr>,<chr>
NDUFS3,NDUFS3,NDUFS3
PDK3,PDK3,PDK3
NDUFA12,NDUFA12,NDUFA12
SLC25A11,SLC25A11,SLC25A11
NDUFS8,NDUFS8,NDUFS8
NDUFS1,NDUFS1,NDUFS1
SLC25A4;SLC25A5;SLC25A6,SLC25A4,SLC25A4
NDUFA13,NDUFA13,NDUFA13
NDUFS2,NDUFS2,NDUFS2


In [31]:
EWCE_specificity_matrix <- read.table("Cell_type_deconvolution/EWCE_Darm&Hodge_specificitymatrix.csv",
                                      header =TRUE, stringsAsFactors=F, sep = ",")

In [32]:
rownames(EWCE_specificity_matrix) <- EWCE_specificity_matrix$Genes

In [33]:
EWCE_specificity_matrix <- EWCE_specificity_matrix[,-1]

In [34]:
EWCE_specificity_matrix

,Astrocytes,Endothelial,ExNeurons,InNeurons,Microglia,Oligodendrocytes,OPC,Unknown
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
X3.8.1.2,0.00000000,0.000000000,0.06802946,0.931942533,0.00000000,0.000000000,0.000000000,0.0000000000
X3.8.1.3,0.87848240,0.000000000,0.07530923,0.046207127,0.00000000,0.000000000,0.000000000,0.0000000000
X3.8.1.4,0.00000000,0.000000000,0.24498483,0.755006814,0.00000000,0.000000000,0.000000000,0.0000000000
X3.8.1.5,0.00000000,0.000000000,0.63114994,0.097241006,0.00000000,0.271498008,0.000000000,0.0000000000
X5.HT3C2,0.06982341,0.000000000,0.18086182,0.225758009,0.01349505,0.037695567,0.472365751,0.0000000000
A1BG,0.01156738,0.000000000,0.20158860,0.125844117,0.10621237,0.553589011,0.001198491,0.0000000000
A1BG.AS1,0.07093724,0.000000000,0.36894088,0.241280229,0.02195090,0.295645028,0.001245377,0.0000000000
A1CF,0.09884967,0.000000000,0.22841492,0.130812357,0.06467911,0.051763628,0.425480171,0.0000000000
A2M,0.03423939,0.189783442,0.01314316,0.009476242,0.71128111,0.018847035,0.023229623,0.0000000000


In [35]:
sct.data <- NULL

In [37]:
sct.data[[1]] <- list(annot=NULL, mean_exp = EWCE_specificity_matrix,
                      median_exp=EWCE_specificity_matrix,
                      specificity=EWCE_specificity_matrix, median_specificity=EWCE_specificity_matrix,
                      specificity_quantiles=EWCE_specificity_matrix, plotting=NULL)

In [48]:
reps=10000 # <- Use 100 bootstrap lists so it runs quickly, for publishable analysis use >10000
level=1 # <- Use level 1 annotations (i.e. Interneurons)

In [49]:
full_results = bootstrap.enrichment.test(sct_data=sct.data, 
                                         hits=cluster_1$HUGO_gene, bg = cell_type_matrix$HUGO_gene, 
                                         genelistSpecies="human",sctSpecies="human",
                                         reps=reps, annotLevel=level)

 [1] "NDUFS3"   "PDK3"     "NDUFA12"  "SLC25A11" "NDUFS8"   "NDUFS1"  
 [7] "SLC25A4"  "NDUFA13"  "NDUFS2"   "NDUFA5"   "NNT"      "LRPPRC"  
[13] "NDUFB3"   "APOO"     "COX4I1"   "NDUFA11"  "SLC25A4"  "SLC25A5" 
[19] "NDUFA9"   "FXYD7"    "NDUFAF4"  "NDUFB8"   "ACTN1"    "SLC25A27"
[25] "ADCY1"    "MTX2"     "NDUFB4"   "NDUFB9"   "GABRB1"   "NDUFA2"  
[31] "GK"       "COX7A2L"  "SLC25A3"  "NDUFC2"   "AP3M1"    "SCN1A"   
[37] "DIP2A"    "BDH1"     "RPL15"    "MPC2"     "DLAT"     "VDAC1"   
[43] "PDHB"     "NDUFB10"  "HSPA12A"  "SLC25A5"  "ATP1A1"   "NDUFA10" 
[49] "NDUFS7"   "SFXN1"    "SLC25A4"  "ATP2B2"   "RAP2A"    "TMEM126A"
[55] "MGST3"    "AGK"      "UQCR10"   "COX5A"    "PRKCB"    "NDUFA4"  
[61] "COX7A2"   "AP2M1"    "RPL5"     "NDUFB1"   "SLC25A12" "CEND1"   
[67] "C6orf136" "GABRG1"   "ABHD16A"  "AP2S1"   
[1] "Astrocytes"
[1] 0.8316
[1] ""
[1] "Endothelial"
[1] 0.4935
[1] ""
[1] "ExNeurons"
[1] 0.0383
[1] "Fold enrichment: 1.12854856043976"
[1] "Standard deviations from me

In [50]:
print(full_results$results)

                         CellType annotLevel      p fold_change sd_from_mean
Astrocytes             Astrocytes          1 0.8316   0.9032411   -0.9623318
Endothelial           Endothelial          1 0.4935   0.9885678   -0.0776509
ExNeurons               ExNeurons          1 0.0383   1.1285486    1.8296027
InNeurons               InNeurons          1 0.0275   1.1207127    1.9823975
Microglia               Microglia          1 0.9994   0.7427034   -2.7077088
Oligodendrocytes Oligodendrocytes          1 0.9286   0.8673244   -1.4059691
OPC                           OPC          1 0.8347   0.9263185   -0.9699301
Unknown                   Unknown          1 0.9637   0.7057488   -1.6990032


In [51]:
## test and re-produce the result from Suzanne